In [1]:
from pipeline import *
import pandas as pd

In [2]:
year = 10

df_teams, df_teams_post, df_series_post, df_players, df_players_teams, df_coaches, df_awards_players = load_data()

df_teams_merged = global_merge(df_teams, df_teams_post, df_series_post,
                                   df_players, df_players_teams, df_coaches, df_awards_players, year)

df_teams_merged.head(20)

c:\Users\ricar\OneDrive\Ambiente de Trabalho\ac\funcs\statistical_analysis.py:136: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_coaches['win_percentage_coach'] = (df_coaches['won'] + df_coaches['post_wins']) / (df_coaches['won'] +


,tmID,year,offensive_strength,defensive_strength,confID,playoff,team_stats,win_percentage_coach,awards
0,ATL,9,0.174422,0.149886,EA,0,0.000000,-2.235235,2
1,ATL,10,0.577290,0.617877,EA,1,0.849232,-2.235235,5
2,CHA,1,0.079772,0.064578,EA,0,0.000000,-1.295938,2
3,CHA,2,0.673662,0.656730,EA,1,-0.243402,0.533667,3
4,CHA,3,0.682665,0.659045,EA,1,-0.681351,0.533667,3
5,CHA,4,0.814721,0.825337,EA,1,-0.473173,-0.478402,3
6,CHA,5,0.889861,0.882206,EA,0,-0.259032,-0.478402,2
7,CHA,6,1.265729,1.413766,EA,0,-0.524261,0.000000,3
8,CHA,7,0.798807,0.972582,EA,0,-0.106067,-0.857600,1
9,CHI,7,0.483068,0.573005,EA,0,0.000000,-2.026503,0
